In [ ]:
import datetime as dt
from datetime import datetime
import sys
import os
import boto3
import pandas as pd
from boto3.session import Session

from io import StringIO
import numpy as np
import shutil

# declaring some constants
bucket = 'arable-adse-dev'
session = Session()
client = session.client('s3')

import json
import psycopg2 as pg
import pandas.io.sql as psql
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# get access to production data
client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId='alp-datasci')
a = json.loads(response['SecretString'])

def get_data_prod(query):
    connection = pg.connect(user = a['alp_user'],
                            password = a['alp_pword'],
                            host = a['alp_host'],
                            port = a['alp_port'],
                            database = a['alp_db'])

    df = psql.read_sql('''%s''' % (query), connection)

    connection.close()
    
    return df


client = boto3.client('secretsmanager')
response = client.get_secret_value(SecretId='han-db')
b = json.loads(response['SecretString'])

def get_data_test(query):    
    connection = pg.connect(user = b['user'],
                            password = b['alt_pword'],
                            host = b['alt_host'],
                            port = b['alt_port'],
                            database = b['alt_db'])

    df = psql.read_sql('''%s''' % (query), connection)

    connection.close()
    
    return df

In [ ]:
# client = boto3.client('secretsmanager')
# response = client.get_secret_value(SecretId='alp-datasci')
# ds = json.loads(response['SecretString'])

# def get_data_prod(query):
#     connection = pg.connect(user = ds['alp_user'],
#                             password = ds['alp_pword'],
#                             host = ds['alp_host'],
#                             port = ds['alp_port'],
#                             database = ds['alp_db'])

#     df = psql.read_sql('''%s''' % (query), connection)

#     connection.close()
    
#     return df

In [ ]:
device = 'C009846'
start = '2022-03-31'
end = '2022-06-23'

#### grab 5min and hourly data mark device

In [ ]:
mark_5min = get_data_prod(f"""
        select c.*, r.lw_temp_3
        from device_data.calibrated c
        join device_data.raw r
        using(time, device)
        where c.device = '{device}'
        and time >= '{start}'
        and time <= '{end}'
        """)

## hourly data
hourly = get_data_prod(f"""
        select time, device, precip, rh
        from device_data.hourly
        where device = '{device}'
        and time >= '{start}'
        and time <= '{end}'
        """)

In [ ]:
mark_5min.head()

In [ ]:
hourly = hourly.rename(columns={'rh':'rh_hourly'})

### join 5min and hourly data

In [ ]:
mark_5min['time'] = mark_5min['time'].astype('datetime64[ns]')
mark_5min['hour'] = mark_5min['time'].dt.floor('h')

#hourly['time'] = hourly['time'].astype('datetime64[ns]')
#hourly['hour'] = hourly['time'].dt.floor('h')
hourly = hourly[['hour', 'precip', 'rh_hourly']]
mark = mark_5min.merge(hourly, on='hour', how='left')

In [ ]:
mark

#### calculate lfw

In [ ]:
# 1 and 2
def get_lfw_current(df, which_tbelow):
    """Takes in df that have calibrated data, lw_temp3 from raw and outer joined with hourly precip
    Can be used to calculate lfw in 1 and 2, which 1 using tbelow instead of lw_temp_3
    """
    tdew_offset = 0.99
    low_rh_threshold = 0.90
    high_rh_threshold = 0.93
    
    if which_tbelow == 'lw_temp_3':
        # make tbelow_V1
        df['tbelow_corr'] = make_tbelow_v1(df.lw_temp_3)
        # make alter dpd with default offsets
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow_corr', tdew_offset, low_rh_threshold)
    elif which_tbelow == 'tbelow':
        # make alter dpd
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow', tdew_offset, low_rh_threshold)
    else:
        raise ValueError(f'not expecting tbelow type to be {which_tbelow}') 
    
    # cooperate ^ with hourly data to make final lfw
    return (df.precip > 0) | (df.rh_hourly > high_rh_threshold) | (df.meta_lfw > 0)

def make_tbelow_v1(sr):
    """Takes lw_temp_3 and returns Tbelow_corr"""
    za = np.array([0.01935698, 0.8618187, -2.34398788])
    return za[0] * (sr ** 2) + za[1] * sr + za[2]

def alter_dew_point_depression(df, tbelow_type, tdew_offset, low_rh_threshold):
    return (df[tbelow_type] <= df.tdew + tdew_offset) \
                    & (df.rh > low_rh_threshold)

In [ ]:
tdf =mark.copy()
tdf['lfw'] = get_lfw_current(tdf, 'lw_temp_3')
tdf.tail()

In [ ]:
tdf =mark.copy()
tdf['lfw'] = get_lfw_current(tdf, 'tbelow')
tdf.tail()

In [3]:
#device = 'C009846'

In [8]:
# df = get_data_prod(f"""
#         select * from device_data.calibrated
#         where device = '{device}'
#         """)

In [13]:
df.tail()

,time,device,location,b1dw,b1uw,b2dw,b2uw,b3dw,b3uw,b4dw,...,tbelow,tdew,dqs,lat,long,create_time,tair_version,tbelow_version,radcal_version,update_time
34832,2022-06-23 16:21:19+00:00,C009846,62b0e0d20f3d0c000e82f72d,1.364267,0.024033,1.815462,0.058323,1.387576,0.058419,1.467102,...,29.55,22.586409,None,10.28583,-83.52443,2022-06-23 16:45:46+00:00,20200128_1,None,20211208160343,2022-06-23 16:45:53+00
34833,2022-06-23 16:26:19+00:00,C009846,62b0e0d20f3d0c000e82f72d,0.893973,0.016108,1.152872,0.037543,0.886458,0.037153,0.918251,...,25.37,22.642298,None,10.28583,-83.52443,2022-06-23 16:45:46+00:00,20200128_1,None,20211208160343,2022-06-23 16:45:53+00
34834,2022-06-23 15:56:18+00:00,C009846,62b0e0d20f3d0c000e82f72d,0.439401,0.008573,0.533835,0.018091,0.434458,0.018454,0.419755,...,26.76,23.844526,None,10.28583,-83.52443,2022-06-23 16:45:42+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 16:46:15+00
34835,2022-06-23 16:01:18+00:00,C009846,62b0e0d20f3d0c000e82f72d,0.358582,0.007730,0.421782,0.014679,0.355226,0.014451,0.329544,...,27.45,23.702810,None,10.28583,-83.52443,2022-06-23 16:45:42+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 16:46:15+00
34836,2022-06-23 16:06:19+00:00,C009846,62b0e0d20f3d0c000e82f72d,0.761297,0.013834,0.977182,0.031898,0.776235,0.032398,0.784878,...,28.78,24.030708,None,10.28583,-83.52443,2022-06-23 16:45:42+00:00,20200128_1,backup: lw_temp_3,20211208160343,2022-06-23 16:46:15+00


In [6]:
# 1 and 2
def get_lfw_current(df, which_tbelow):
    """Takes in df that have calibrated data, lw_temp3 from raw and outer joined with hourly precip
    Can be used to calculate lfw in 1 and 2, which 1 using tbelow instead of lw_temp_3
    """
    tdew_offset = 0.99
    low_rh_threshold = 0.90
    high_rh_threshold = 0.93
    
    if which_tbelow == 'lw_temp_3':
        # make tbelow_V1
        df['tbelow_corr'] = make_tbelow_v1(df.lw_temp_3)
        # make alter dpd with default offsets
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow_corr', tdew_offset, low_rh_threshold)
    elif which_tbelow == 'tbelow':
        # make alter dpd
        df['meta_lfw'] = alter_dew_point_depression(df, 'tbelow', tdew_offset, low_rh_threshold)
    else:
        raise ValueError(f'not expecting tbelow type to be {which_tbelow}') 
    
    # cooperate ^ with hourly data to make final lfw
    return (df.precip > 0) | (df.rh_hourly > high_rh_threshold) | (df.meta_lfw > 0)

def make_tbelow_v1(sr):
    """Takes lw_temp_3 and returns Tbelow_corr"""
    za = np.array([0.01935698, 0.8618187, -2.34398788])
    return za[0] * (sr ** 2) + za[1] * sr + za[2]

def alter_dew_point_depression(df, tbelow_type, tdew_offset, low_rh_threshold):
    return (df[tbelow_type] <= df.tdew + tdew_offset) \
                    & (df.rh > low_rh_threshold)

In [16]:
# tdf =df.copy()
# tdf['lfw'] = get_lfw_current(tdf, 'lw_temp_3')
# tdf.head()